# NodeMCU_20 Data, Cleaning of Data

# 1. Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 2. Reading WiSenseData

In [2]:
dff = pd.read_csv('NodeMCU_020.csv' , header = None)  #reading the Outdoor dataset

In [3]:
#There are 4 columns
dff.columns

Int64Index([0, 1, 2, 3], dtype='int64')

# 3. Renaming the Columns

In [4]:
df = dff.rename(columns={0: 'datestamp', 1: 'NodeAdd' , 2: 'Temperature', 3: 'Humidity'})

In [5]:
df.head() # First 5 rows

,datestamp,NodeAdd,Temperature,Humidity
0,2019-01-03 23:00:00,IoT/NodeMCU_20/dht22/hum,13,45
1,2019-01-03 23:03:07,IoT/NodeMCU_20/dht22/hum,13,45
2,2019-01-03 23:29:56,IoT/NodeMCU_20/dht22/hum,12,44
3,2019-01-03 23:30:04,IoT/NodeMCU_20/dht22/hum,12,44
4,2019-01-03 23:35:22,IoT/NodeMCU_20/dht22/hum,12,44


In [6]:
# Getting some info about Dataset
df.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17555 entries, 0 to 17554
Data columns (total 4 columns):
datestamp      17555 non-null object
NodeAdd        17555 non-null object
Temperature    17555 non-null int64
Humidity       17555 non-null int64
dtypes: int64(2), object(2)
memory usage: 548.7+ KB


In [7]:
#Copying the original dataset ('df') into data1
data1 = df.copy() 

In [8]:
#Describing different features of DataSet
data1.describe() 

,Temperature,Humidity
count,17555.000000,17555.000000
mean,19.636286,47.717345
std,4.882688,10.094002
min,8.000000,9.000000
25%,15.000000,42.000000
50%,19.000000,49.000000
75%,24.000000,55.000000
max,32.000000,71.000000


In [9]:
#Printing first five rows of dataset
data1.head()

,datestamp,NodeAdd,Temperature,Humidity
0,2019-01-03 23:00:00,IoT/NodeMCU_20/dht22/hum,13,45
1,2019-01-03 23:03:07,IoT/NodeMCU_20/dht22/hum,13,45
2,2019-01-03 23:29:56,IoT/NodeMCU_20/dht22/hum,12,44
3,2019-01-03 23:30:04,IoT/NodeMCU_20/dht22/hum,12,44
4,2019-01-03 23:35:22,IoT/NodeMCU_20/dht22/hum,12,44


In [10]:
#Getting total number of Unique Nodes from the DataSet
data1.groupby('NodeAdd').count() 

,datestamp,Temperature,Humidity
NodeAdd,,,
IoT/NodeMCU_20/dht22/hum,68,68,68
NodeMCU_20,17487,17487,17487


In [11]:
#Converting datatype of 'timeStamp' to datetime type
data1['datestamp'] = pd.to_datetime(data1['datestamp'])  

In [12]:
#Now we can se the data type of 'timeStamp', it is now datetime type
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17555 entries, 0 to 17554
Data columns (total 4 columns):
datestamp      17555 non-null datetime64[ns]
NodeAdd        17555 non-null object
Temperature    17555 non-null int64
Humidity       17555 non-null int64
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 548.7+ KB


In [13]:
# Now We will create a new columns in our Dataset namely, 'temp1_changed'.
# This column will contain value '1' if temperature1 is changed else it will contain 0
data1['temp_changed'] = 0

In [14]:
# Grouping the DataSet by 'temp3_changed'
data1.groupby('temp_changed').count()

,datestamp,NodeAdd,Temperature,Humidity
temp_changed,,,,
0,17555,17555,17555,17555


In [15]:
#Checking Outliers
'''
# Following Scripts will deal with first value of each node if it is outlier

1. We'll just check if first value of each node for a particular column is outlier (i.e temperature > 100 or temperature < 0), if it is outlier then we'll change its value to next row value

'''


from tqdm import tqdm_notebook

nodes = data1['NodeAdd'].unique() # this line will create an array having total unique nodes

print('Checking Outlier for Temperature')
for n in tqdm_notebook(nodes , desc = 'Processing records for Outlier'):
    for i in range(data1.shape[0] - 1):
        if(data1.loc[i , 'NodeAdd'] == n):
            val0 = float(data1.loc[i,'Temperature'])
            if(val0 < 0 or val0 > 100):
                data1.loc[i,'Temperature'] = data1.loc[i+1,'Temperature']
                print('Outlier Found at', i , 'for node' , n)
                break
            else:
                break

Checking Outlier for Temperature


# Following is the function to clean 'temperature1'

# Logic behind cleaning the data

Example: Cleaning temperature1

To clean 'temperature1', we'll iterate through this column and select two values(rows) of a particular node and compare it.
1. If there absolute difference is more than 10C and timeinterval is less than 30 minutes then we'll replace later value with previous one.
2. If later value(row) is showing an Outlier and time interval is more than 30 minutes then will just replace it with 'NaN'.




In [22]:
from tqdm import tqdm_notebook

nodes = data1['NodeAdd'].unique() # this line will create an array having total unique nodes

#Function to clean 'temperature'

def temperature_clean(df):
    for n in tqdm_notebook(nodes , desc = 'Processing records'):
    #for n in nodes:
        k = 0
        for i in range(k , df.shape[0]-1):
          if(df.loc[i, 'NodeAdd'] == n):
            val0 = float(df.loc[i,'Temperature'])
            time0 = (df.loc[i,'datestamp' ])
            for j in range(i+1, df.shape[0]-1):
              if(df.loc[j, 'NodeAdd'] == n):
                val1 = float(df.loc[j , 'Temperature'])
                time1 = (df.loc[j , 'datestamp'])
                timedelta = time1 - time0
                minutes = timedelta.total_seconds() / 60
                
                if (abs(val1 - val0) > 10 and minutes < 30.0):
                  df.loc[j,'Temperature'] = val0
                  df.loc[j, 'temp_changed'] = 1
                  k = j
                  break
                elif(((val1) > 100 or (val1) < 0 ) and minutes > 30.0):
                  df.loc[j,'Temperature'] = 'NaN'
                  k = j
                  break
                else:
                  k = j
                  break
                    
                    
                    

In [23]:
#Call Above function to clean the dataset
print('Cleaning Temperature')
temperature_clean(data1)

Cleaning Temperature


In [24]:
data1.groupby('temp_changed').count()

,datestamp,NodeAdd,Temperature,Humidity
temp_changed,,,,
0,17555,17555,17555,17555


In [25]:
df.describe()

,Temperature,Humidity
count,17555.000000,17555.000000
mean,19.636286,47.717345
std,4.882688,10.094002
min,8.000000,9.000000
25%,15.000000,42.000000
50%,19.000000,49.000000
75%,24.000000,55.000000
max,32.000000,71.000000
